In [1]:
import math

def estimate_slurm_resources(sim, n_inhom: int, n_times: int, n_batches: int) -> tuple[str, str]:
    """Estimate SLURM memory and time requirements based on workload."""
    combos = n_times * n_inhom
    num_combos_per_batch = combos // n_batches
    len_coh_times = n_times  # coherence times
    
    # Estimate memory: base 0.3G + factor for data size (complex64 = 8 bytes)
    len_t = len(sim.times_local)
    mem_gb = 0.3 + (num_combos_per_batch * len_t * 8 * 30) / (1024**3)
    requested_mem_gb = math.ceil(mem_gb * 10) / 10
    requested_mem = f"{requested_mem_gb}G"
    
    # Estimate time: scale based on solver, n_atoms, len_coh_times
    solver = sim.simulation_config.ode_solver
    n_atoms = sim.system.n_atoms
    base_time_per_combo_seconds = 5.78e-6  # normalized from example: 50 combos in 26s for ME with 1 atom, ~300 t_det points
    if solver == "BR":
        base_time_per_combo_seconds *= 20  # safety factor for BR
    base_time_per_combo_seconds *= n_atoms ** 2  # quadratic scaling with n_atoms (matrix diagonalization)
    base_time_per_combo_seconds *= len_t * len_coh_times  # quadratic scaling with detection time length
    time_seconds = num_combos_per_batch * base_time_per_combo_seconds
    time_hours = max(0.1, time_seconds / 3600)  # minimum ~36 seconds for safety
    if time_hours < 1:
        minutes = int(time_hours * 60)
        requested_time = f"00:{minutes:02d}:00"
    else:
        days = int(time_hours) // 24
        hours = int(time_hours) % 24
        if days > 0:
            requested_time = f"{days}-{hours:02d}:00:00"
        else:
            requested_time = f"{hours:02d}:00:00"
    
    return requested_mem, requested_time

In [2]:
# Mock simulation object for testing
class MockSim:
    def __init__(self, ode_solver, n_atoms, len_t=1000):
        self.simulation_config = type('obj', (object,), {'ode_solver': ode_solver})
        self.system = type('obj', (object,), {'n_atoms': n_atoms})
        self.times_local = [0] * len_t  # mock times_local array

In [5]:
test_cases = [
    ('1D ME, 1 atom', MockSim('ME', 1), 500, 1, 10),
    ('1D BR, 1 atom', MockSim('BR', 1), 500, 1, 10),
    ('1D BR, 2 atoms', MockSim('BR', 2), 500, 1, 10),
    ('2D (n_times=100) ME, 1 atom', MockSim('ME', 1), 500, 100, 10),
    ('2D (n_times=100) ME, 1 atom', MockSim('ME', 1), 500, 100, 20),
    ('2D (n_times=500) ME, 1 atom', MockSim('ME', 1), 500, 500, 10),
    ('2D (n_times=500) BR, 1 atom', MockSim('BR', 1), 500, 500, 10),
    ('2D (n_times=500) BR, 2 atoms', MockSim('BR', 2), 500, 500, 10),
]

In [6]:
# Run tests
print("SLURM Resource Estimation Test Results")
print("=" * 60)
for desc, sim, n_inhom, n_times, n_batches in test_cases:
    mem, time = estimate_slurm_resources(sim, n_inhom, n_times, n_batches)
    combos = n_times * n_inhom
    combos_per_batch = combos // n_batches
    print(f"{desc}:")
    print(f"  Total combos: {combos}, Combos/batch: {combos_per_batch}, N_inhom: {n_inhom}, N_times: {n_times}, N_batches: {n_batches}")
    print(f"  Solver: {sim.simulation_config.ode_solver}, N_atoms: {sim.system.n_atoms}")
    print(f"  Estimated Memory: {mem}, Time: {time}")
    print()

SLURM Resource Estimation Test Results
1D ME, 1 atom:
  Total combos: 500, Combos/batch: 50, N_inhom: 500, N_times: 1, N_batches: 10
  Solver: ME, N_atoms: 1
  Estimated Memory: 0.4G, Time: 00:06:00

1D BR, 1 atom:
  Total combos: 500, Combos/batch: 50, N_inhom: 500, N_times: 1, N_batches: 10
  Solver: BR, N_atoms: 1
  Estimated Memory: 0.4G, Time: 00:06:00

1D BR, 2 atoms:
  Total combos: 500, Combos/batch: 50, N_inhom: 500, N_times: 1, N_batches: 10
  Solver: BR, N_atoms: 2
  Estimated Memory: 0.4G, Time: 00:06:00

2D (n_times=100) ME, 1 atom:
  Total combos: 50000, Combos/batch: 5000, N_inhom: 500, N_times: 100, N_batches: 10
  Solver: ME, N_atoms: 1
  Estimated Memory: 1.5G, Time: 00:48:00

2D (n_times=100) ME, 1 atom:
  Total combos: 50000, Combos/batch: 2500, N_inhom: 500, N_times: 100, N_batches: 20
  Solver: ME, N_atoms: 1
  Estimated Memory: 0.9G, Time: 00:24:00

2D (n_times=500) ME, 1 atom:
  Total combos: 250000, Combos/batch: 25000, N_inhom: 500, N_times: 500, N_batches: 10

## Test Results Explanation

All tests use len_t=1000 (detection time points), num_combos_per_batch=50.

- **1D cases** (len_coh_times=1): Fast, minimal scaling.
- **2D cases**: Scale with len_coh_times (100 or 500).
- **BR vs ME**: BR has 20x time factor.
- **Atom count**: Quadratic scaling with n_atoms.

Adjust base values and factors based on real benchmarks!

# Testing SLURM Resource Estimation

This notebook tests the `estimate_slurm_resources` function with various parameter combinations to understand expected memory and time requirements.